In [1]:
import pandas as pd

In [2]:

file_path = '../../data/processed/feature_selected_data.parquet'
df = pd.read_parquet(file_path, engine= 'pyarrow')

In [3]:
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'durata_erogazione_min', 'eta'],
      dtype='object')

In [4]:
df['data_erogazione'] = pd.to_datetime(df['data_erogazione'], utc=True)

df['anno'] = df['data_erogazione'].dt.year
df['quadrimestre'] = df['data_erogazione'].dt.quarter


In [5]:
df.head()

,id_prenotazione,id_paziente,data_nascita,sesso,regione_residenza,asl_residenza,provincia_residenza,comune_residenza,tipologia_servizio,descrizione_attivita,...,struttura_erogazione,codice_struttura_erogazione,tipologia_struttura_erogazione,id_professionista_sanitario,tipologia_professionista_sanitario,data_erogazione,durata_erogazione_min,eta,anno,quadrimestre
0,JUOCPQCUG1H96QO,LCWFXF28X79P382V,1980-04-24 00:00:00+00:00,male,Piemonte,CN1,Cuneo,Fossano,Teleassistenza,Insegnamento al paziente e/o alla sua famiglia...,...,AZ. OSPEDAL. S. CROCE E CARLE,10906,Azienda Ospedaliera,MZRAER00D13A287J,Infermiere,2019-02-04 10:00:00+00:00,2925.0,44,2019,1
1,XDP99NXRHPOYFYU,LRFZOI94E58U012I,1972-01-30 00:00:00+00:00,female,Toscana,AZIENDA USL TOSCANA CENTRO,Firenze,Calenzano,Teleassistenza,Prestazione infermieristica connessa alla nece...,...,AZIENDA OSPEDALIERA MEYER,90904,Azienda Ospedaliera Universitaria,EBCRPU12T24U316I,Infermiere,2019-02-01 10:00:00+00:00,1895.0,52,2019,1
2,6VYI0AK57ECO5WU,JOBCGJ98A04B765H,1986-06-27 00:00:00+00:00,male,Emilia romagna,AZIENDA USL BOLOGNA,Bologna,Bologna,Teleassistenza,Prestazione dietistica,...,PRESIDIO OSPEDALIERO UNICO - AZIENDA DI,80053,Ospedale a gestione diretta,PMEJNL62M27F357I,Dietista,2019-02-03 06:00:00+00:00,1165.0,38,2019,1
3,VLFS5J9E3Q74Q2W,TKROSE51Z77Q065E,1967-10-21 00:00:00+00:00,female,Campania,A.S.L. NAPOLI 1 CENTRO,Napoli,Napoli,Teleassistenza,Psicoterapia individuale,...,A.O.R.N. SANTOBONO-PAUSILIPON,150902,Azienda Ospedaliera,VNNRMQ15C69W200K,Psicologo,2019-02-01 06:00:00+00:00,1969.0,56,2019,1
4,XD8SZX9JOVCMEXW,WRRVSK73Z45L640S,1963-08-24 00:00:00+00:00,male,Abruzzo,PESCARA,Pescara,Pescara,Teleassistenza,Prestazione dietistica,...,P.O. 'SPIRITO SANTO' PESCARA,130018,Ospedale a gestione diretta,GIQCNR34T04P154U,Dietista,2019-02-05 14:00:00+00:00,3099.0,61,2019,1


In [24]:
grouped = df.groupby(['anno', 'quadrimestre', 'regione_erogazione']).size().reset_index(name='num_servizi')

In [25]:
grouped.sort_values(by=['num_servizi'], ascending=True)

,anno,quadrimestre,regione_erogazione,num_servizi
19,2019,1,Valle d`aosta,23
40,2019,2,Valle d`aosta,31
12,2019,1,Prov. auton. bolzano,43
10,2019,1,Molise,43
313,2022,3,Valle d`aosta,44
...,...,...,...,...
218,2021,3,Lombardia,5475
197,2021,2,Lombardia,5554
134,2020,3,Lombardia,5735
176,2021,1,Lombardia,6785


In [26]:
# Calcolo dell'incremento per ogni quadrimestre
grouped['incremento'] = grouped.groupby(['quadrimestre', 'regione_erogazione'])['num_servizi'].diff()


In [27]:
grouped.sort_values(by=['incremento'], ascending=False)

,anno,quadrimestre,regione_erogazione,num_servizi,incremento
155,2020,4,Lombardia,7055,3297.0
92,2020,1,Lombardia,4379,2883.0
176,2021,1,Lombardia,6785,2406.0
153,2020,4,Lazio,5024,2285.0
150,2020,4,Campania,4808,2099.0
...,...,...,...,...,...
79,2019,4,Sicilia,2259,NaN
80,2019,4,Toscana,784,NaN
81,2019,4,Umbria,328,NaN
82,2019,4,Valle d`aosta,54,NaN


In [28]:
#TODO: eliminare i valori nulli non va bene, anzi vuol dire che quelle tipo di prenotazioni non sono mai stati fatte in quel quadrimestre

grouped = grouped.dropna(subset=['incremento'])

In [39]:
#TODO: COME VEDI LA MEDIA NON VA BENE, va usato l'incremento percentuale
grouped.sort_values(by=['incremento'], ascending=False).sort_values(by=['quadrimestre', 'regione_erogazione', 'anno'])[grouped['regione_erogazione'] == 'Valle d`aosta']

/tmp/ipykernel_661847/2968197338.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  grouped.sort_values(by=['incremento'], ascending=False).sort_values(by=['quadrimestre', 'regione_erogazione', 'anno'])[grouped['regione_erogazione'] == 'Valle d`aosta']


,anno,quadrimestre,regione_erogazione,num_servizi,incremento,incremento_medio
103,2020,1,Valle d`aosta,64,41.0,13.000000
187,2021,1,Valle d`aosta,118,54.0,13.000000
271,2022,1,Valle d`aosta,62,-56.0,13.000000
124,2020,2,Valle d`aosta,82,51.0,11.666667
208,2021,2,Valle d`aosta,71,-11.0,11.666667
292,2022,2,Valle d`aosta,66,-5.0,11.666667
145,2020,3,Valle d`aosta,93,44.0,-1.666667
229,2021,3,Valle d`aosta,68,-25.0,-1.666667
313,2022,3,Valle d`aosta,44,-24.0,-1.666667
166,2020,4,Valle d`aosta,104,50.0,1.666667


In [30]:
mask = (grouped['quadrimestre'] == 4) & (grouped['regione_erogazione'] == 'Lombardia')

grouped[mask]

,anno,quadrimestre,regione_erogazione,num_servizi,incremento
155,2020,4,Lombardia,7055,3297.0
239,2021,4,Lombardia,4911,-2144.0
323,2022,4,Lombardia,4532,-379.0


In [31]:
grouped['incremento_medio'] = grouped.groupby(['quadrimestre', 'regione_erogazione'])['incremento'].transform('mean')

In [32]:
grouped['incremento_medio'].quantile([0.25, 0.5, 0.75])

0.25     22.166667
0.50     67.333333
0.75    172.666667
Name: incremento_medio, dtype: float64

In [33]:
mask = (grouped['quadrimestre'] == 4) & (grouped['regione_erogazione'] == 'Lombardia')

grouped[mask]

,anno,quadrimestre,regione_erogazione,num_servizi,incremento,incremento_medio
155,2020,4,Lombardia,7055,3297.0,258.0
239,2021,4,Lombardia,4911,-2144.0,258.0
323,2022,4,Lombardia,4532,-379.0,258.0


In [16]:
# grouped['incremento_medio'] = grouped.groupby(['quadrimestre', 'descrizione_attivita', 'struttura_erogazione'])['incremento'].transform('mean')

In [17]:
# Q1, Q2, Q3 = grouped['incremento_medio'].quantile([0.25, 0.5, 0.75])

In [18]:
# print(Q1)
# print(Q2)
# print(Q3)


In [19]:
# def classify_increment(value):

#     if value <= Q1:
#         return 'Low Increment'
#     elif value <= Q2:
#         return 'Medium Increment'
#     elif value <= Q3:
#         return 'Costant Increment'
#     else:
#         return 'High Increment'


In [20]:
# grouped['incremento_teleassistenze'] = grouped['incremento_medio'].apply(classify_increment)

In [21]:
# grouped